# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 1 ноября 2018, 06:00 
**Штраф за опоздание:** -2 балла после 06:00 1 ноября, -4 балла после 06:00 8 ноября, -6 баллов после 06:00 15 ноября

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (4 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы fit такой, чтобы она была медленнее sklearn не более чем в 10 раз. Скорость проверяем на  wine и Speed Dating Data. Для ускорения используем только numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 3 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [2]:
#pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
            self.G_f = self.__gini_f
        elif criterion == 'entropy':
            self.G_function = self.__entropy
            self.G_f = self.__entropy_f
        elif criterion == 'misclass':
            self.G_function = self.__misclass
            self.G_f = self.__misclass_f
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - np.sum(l_c ** 2 / l_s + r_c ** 2 / r_s, axis=1)

    def __entropy(self, l_c, l_s, r_c, r_s):
        return - np.sum(l_c * np.log((l_c) / l_s) + r_c * np.log((r_c) / r_s),
                        axis=1).reshape(-1, 1) / (l_s + r_s)

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - np.max(l_c / (l_s + r_s), axis=1) - np.max(
            r_c / (l_s + r_s), axis=1)

    def __gini_f(self, y):
        return 1 - ((np.bincount(y) / y.size) ** 2).sum()

    def __entropy_f(self, y):
        return -((np.bincount(y) / y.size)
                 * np.log2(np.bincount(y) / y.size)).sum()

    def __misclass_f(self, y):
        return 1 - (np.bincount(y) / y.size).max()

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(n_feature))]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_feature))]

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Сортировка по признакам x
        # Получение колличества классов
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = self.num_class
        cut_size = np.int(self.min_samples_split / 2 - 1)

        # Поиск индексов на которых происходит смена класса
        if cut_size == 0:
            splitted_sorted_y = sorted_y
        else:
            splitted_sorted_y = sorted_y[cut_size:-cut_size]
        r_border_ids = np.where(splitted_sorted_y[:-1] !=
                                splitted_sorted_y[1:])[0] + (cut_size + 1)

        if len(r_border_ids) == 0:
            return np.inf, None

        # Кодировка разбиения - 1 - класс поменялся, 0 - не поменялся
        eq_el_count = r_border_ids - np.append(np.array([cut_size]),
                                               r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + \
            np.bincount(sorted_y[:cut_size], minlength=class_number)

        # Подсчет колличества классов и элементов
        # в левой и правой частях разбиения
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y,
                                    minlength=class_number) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Минимизация неопределенности
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        # Возвращение наименьшей неопределенности и порог разделения
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        y_count = np.bincount(y)
        y_max = y_count.argmax()
        node_leaf = self.LEAF_TYPE, y_max, y_count / y.size
        if (y.size >= self.min_samples_split) and (depth != self.max_depth):
            if y_count[y_max] < y.size * self.sufficient_share:
                feature_ids = self.get_feature_ids(x.shape[1])
                temp = np.array([self.__find_threshold(x[:, i], y)
                                 for i in feature_ids])
                best_imp = temp[:, :1].argmin()
                threshold = list(temp[best_imp]) + [best_imp]
                Xleft, Xright, Yleft, Yright =\
                    self.__div_samples(x, y, threshold[2], threshold[1])
                if Xleft.size and Xright.size:
                    self.tree[node_id] =\
                        self.NON_LEAF_TYPE, threshold[2], threshold[1]
                    self.feature_importances_[threshold[2]] +=\
                        self.G_f(y) * y.size - (
                        self.G_f(Yleft) * Yleft.size + self.G_f(Yright)
                        * Yright.size)
                    self.__fit_node(Xleft, Yleft, 2 * node_id + 1,
                                    depth + 1, pred_f)
                    self.__fit_node(Xright, Yright, 2 * node_id + 2,
                                    depth + 1, pred_f)
                else:
                    self.tree[node_id] = node_leaf
            else:
                self.tree[node_id] = node_leaf
        else:
            self.tree[node_id] = node_leaf

    def fit(self, x, y):
        self.feature_importances_ = np.zeros(x.shape[1])
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ /= self.feature_importances_.sum()

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(
    wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [5]:
%time clf.fit(X_train, y_train)

Wall time: 1.97 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)

Wall time: 22.9 ms


## Проверка качества работы на wine

In [7]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.7835497835497836

In [8]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8917748917748917

## Подготовка данных Speed Dating Data 

In [9]:
#%%pycodestyle

df = pd.read_csv('speed-dating-experiment\Speed Dating Data.csv',
                 encoding='cp1251')
df = df.iloc[:, :97]
df = df.drop([
    'id', 'idg', 'condtn', 'round', 'position', 'positin1', 'order', 'partner',
    'age_o', 'race_o', 'pf_o_att', 'pf_o_sin', 'pf_o_int', 'pf_o_fun',
    'pf_o_amb', 'pf_o_sha', 'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
    'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o', 'field', 'undergra',
    'from', 'zipcode', 'career', 'sports', 'tvsports', 'exercise', 'dining',
    'museums', 'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv',
    'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga', 'expnum',
    'wave'], axis=1)
df = df.dropna(subset=['age'])
df = df.dropna(subset=['date'])
df = df.dropna(subset=['imprelig', 'imprace'])
df.field_cd = df.field_cd.fillna(0)
df.career_c = df.career_c.fillna(0)
df.mn_sat = df.mn_sat.str.replace(',', '').astype(np.float)
df.mn_sat = df.mn_sat.fillna(-999)
df.tuition = df.tuition.str.replace(',', '').astype(np.float)
df.tuition = df.tuition.fillna(-999)
df.income = df.income.str.replace(',', '').astype(np.float)
df.income = df.income.fillna(-999)
df.loc[:, 'temp_totalsum'] = df.loc[
    :, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']
].sum(axis=1)
df.loc[
    :, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']
] = (df.loc[
    :, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']
].T/df.temp_totalsum.T).T * 100
df.temp_totalsum = df.loc[
    :, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']
].sum(axis=1)
df.loc[
    :, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']
] = (df.loc[
    :, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']
].T/df.temp_totalsum.T).T * 100
df = df.drop(['temp_totalsum'], axis=1)
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i), 'intel{}_1'.format(i),
            'fun{}_1'.format(i), 'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    df = df.drop(feat, axis=1)
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid']).drop(
    ['gender'], axis=1
).dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid']).drop(
    ['gender', 'match', 'int_corr', 'samerace'], axis=1
).dropna()
df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'), on='pid', how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

## Проверка скорости работы на Speed Dating Data 

In [10]:
clf = DecisionTreeClassifier(min_samples_split=2)
#my_clf = MyDecisionTreeClassifier(min_samples_split=2)

In [11]:
%time clf.fit(X_train, y_train)

Wall time: 89.8 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [12]:
%time my_clf.fit(X_train, y_train)

Wall time: 414 ms


## Проверка качества работы на Speed Dating Data

In [13]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5376514405582431

In [14]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.4945464219207236

## Задание 3

In [15]:
print(pd.Series(
    data=clf.feature_importances_, index=df_pair.columns[1:]
).sort_values(ascending=False).head(10))

int_corr     0.083531
income_f     0.030370
fun2_1_f     0.027294
fun3_1       0.026819
sinc1_1      0.026588
fun1_1       0.024640
attr3_1_f    0.023602
attr2_1      0.022928
sinc2_1_f    0.022819
mn_sat       0.022636
dtype: float64


In [16]:
print(pd.Series(
    data=my_clf.feature_importances_, index=df_pair.columns[1:]
).sort_values(ascending=False).head(10))

int_corr     0.146294
date         0.089366
shar2_1      0.072407
income_f     0.067505
amb1_1_f     0.056589
attr3_1      0.054368
career_c     0.051606
amb3_1_f     0.050611
date_f       0.040735
attr2_1_f    0.036861
dtype: float64


## Задание 4

In [17]:
params = {'max_depth': range(1,15),
          'n_estimators': range(5, 100, 2),
          'min_samples_leaf': range(1,15, 2),
          'criterion': ['gini', 'entropy']}
clf = RandomForestClassifier()

random_search = RandomizedSearchCV(clf, param_distributions=params)
random_search.fit(X, y)
print(random_search.best_params_)

{'n_estimators': 85, 'min_samples_leaf': 11, 'max_depth': 8, 'criterion': 'gini'}
